# Implementação DQN com prioritized replay no ambiente Swift com manipulador panda.

Replay buffer <br>
DNN para representar o agente atual e uma DNN alvo com uma taxa de atualização menor.

In [1]:
import torch
import json
import pickle
import random
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
from IPython.display import clear_output
from scipy.signal import convolve, gaussian
import os
import io
import base64
import time
import glob
from IPython.display import HTML
import data_panda as rbt

from torch.utils.tensorboard import SummaryWriter
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
%matplotlib inline


#j3 range -0.08 a 3.75  #j2 range -0.07 a -3. #j1 range -1.8 a 1.76
torch.cuda.is_available()

True

### Sistema observável e com medidas dos ângulos disponíveis


    Observation:
        Type: Box(4)
        Num     Observation               Min                     Max
        0       Joint1                   -4.8                    4.8
        1       Joint2                    -Inf                    Inf
        2       Joint3                -0.418 rad (-24 deg)    0.418 rad (24 deg)
        
    Actions:
        Type: Discrete(9)
        Num   Three actions for each joint
        0     decrement joint j
        1     increment joint j
        2     decrement join  j

        #j3 range 0.0 a 3.7
        #j2 range 0.0 a -3.
        #j1 range -1.7 a 1.7


        



In [2]:
# self.panda = rp.models.Panda()
# self.panda_end = rp.models.Panda()
# self.m=100 #magnification factor

# self.obstacle = Cuboid([0.2, 0.2, 0.8], pose=sm.SE3(0.3, 0, 0)) 
# #self.floor = Cuboid([0.2, 0.2, 0.8], pose=sm.SE3(-0.2, 0, 0)) 
# self.obs_floor = Cuboid([2., 2., 0.01], pose=sm.SE3(0, 0, 0), color=[100,100,100,0])#"black") #color=[100,100,100,0]
# self.scene.add(self.obs_floor)
# self.scene.add(self.obstacle)
# self.scene.add(self.panda, robot_alpha=0.6)
# self.delta=delta
# #End joints positions
# j=[0.8,-1.5,1] 
# self.q_end=[0., j[0], 0.,j[1], 0., j[2], 0.]
# self.Tep = self.panda.fkine(self.q_end)

In [3]:
state_shape = 3
env=rbt.Panda_RL()
agent=rbt.DQNAgent(state_shape, epsilon=0).to(device)
env.delta=0.02
RESTORE_AGENT=False # Restore a trained agent
NEW_BUFFER=True # Restore a buffer
TRAIN=True # Train or only simulate
env.renderize=True #stop robot viewing

In [4]:

env.panda

ERobot: panda (by Franka Emika), 7 joints (RRRRRRR), 1 gripper, geometry, collision
┌─────┬──────────────┬───────┬─────────────┬────────────────────────────────────────────────┐
│link │     link     │ joint │   parent    │              ETS: parent to link               │
├─────┼──────────────┼───────┼─────────────┼────────────────────────────────────────────────┤
│   0 │ panda_link0  │       │ BASE        │                                                │
│   1 │ panda_link1  │     0 │ panda_link0 │ SE3(0, 0, 0.333) ⊕ Rz(q0)                      │
│   2 │ panda_link2  │     1 │ panda_link1 │ SE3(-90°, -0°, 0°) ⊕ Rz(q1)                    │
│   3 │ panda_link3  │     2 │ panda_link2 │ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2)       │
│   4 │ panda_link4  │     3 │ panda_link3 │ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3)       │
│   5 │ panda_link5  │     4 │ panda_link4 │ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(q4) │
│   6 │ panda_link6  │     5 │ panda_link5 │ SE3(90°, -0°, 0°) ⊕ Rz(q5

In [5]:
target_network = rbt.DQNAgent(agent.state_shape, epsilon=0.5).to(device)
#Copying weights from agent network
target_network.load_state_dict(agent.state_dict())

<All keys matched successfully>

In [6]:
env.panda.q=env.panda.qz
# set a seed
seed = 13
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [7]:
q_far=np.array([ 0., -0.8 ,  0. , -0.0698,  0.,  3.3825,  0.    ])
env.panda.q=q_far
env.scene.step()
print(env.distance(),env.fitness())


7.245333459951897 2.8228782900000007


In [8]:
env.panda.q=env.panda.qz
env.scene.step()
print(env.distance(),env.fitness())

3.0643562098777974 14.428072040000002


In [9]:
env.panda.q=env.q_goal
env.scene.step()
print(env.distance(),env.fitness())
T_end=env.panda.fkine(env.q_goal)
T_end

2.842994640253499 0.0


  -0.6095   -0.6095    0.5069    0.6235    
   0.7071   -0.7071    0         0         
   0.3584    0.3584    0.862     0.9592    
   0         0         0         1         


In [10]:
def set_res_dir():
    # Directory to store results
    res_dir_count = len(glob.glob('runs/*'))
    print(f"Current number of result directories: {res_dir_count}")
    if TRAIN:
        RES_DIR = f"runs/results_{res_dir_count+1}"
        print(RES_DIR)
    else:
        RES_DIR = f"runs/results_{res_dir_count}"
    return RES_DIR

set_res_dir()



Current number of result directories: 0
runs/results_1


'runs/results_1'

### Main loop



In [11]:
# Fill buffer with samples collected ramdomly from environment
buffer_len=5000
tmax=800
env.renderize=False

In [14]:


env.renderize=False
exp_replay = rbt.PrioritizedReplayBuffer(buffer_len)
if NEW_BUFFER:
    
    
    for i in trange(60):
        
        state=env.reset()
        # Play 100 runs of experience with 100 steps and  stop if reach 10**4 samples
        rbt.play_and_record(state, agent, env, exp_replay, n_steps=60)
        
        if len(exp_replay) == buffer_len:
            break
    print(len(exp_replay))



    with open('buffer.pickle', 'wb') as handle:
        pickle.dump(exp_replay.buffer, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
else:
    exp_replay = rbt.PrioritizedReplayBuffer(buffer_len)
    with open('buffer.pickle', 'rb') as handle:
        exp_replay.buffer=pickle.load(handle)



  0%|          | 0/60 [00:00<?, ?it/s]


5000


In [38]:


# def monitor_tensorboard():
#     %reload_ext tensorboard
#     %tensorboard --logdir runs

In [15]:
RES_DIR = set_res_dir()
comment=""
# monitor_tensorboard()
tb=SummaryWriter(log_dir=RES_DIR, comment=comment)

Current number of result directories: 0
runs/results_1


In [17]:
percentage_of_total_steps=0.85

In [18]:
#setup some parameters for training
timesteps_per_epoch = 1
batch_size = 64
total_steps = 40* 10**3
#total_steps = 10

#init Optimizer
lr=1e-4
opt = torch.optim.Adam(agent.parameters(), lr=lr)

# set exploration epsilon 
start_epsilon = 1
start_epsilon = 0.1
end_epsilon = 0.05
eps_decay_final_step = percentage_of_total_steps*total_steps

# setup some frequency for logging and updating target network
loss_freq = 40
refresh_target_network_freq = 50
eval_freq = 1000

# to clip the gradients
max_grad_norm = 5000

hyperparameters_train={"start_epsilon":start_epsilon,
                        "end_epsilon":end_epsilon,
                        "lr":lr,
                        "batch_size":batch_size,
                        "total_steps":total_steps,
                        "percentage_of_total_steps":percentage_of_total_steps,
                        "refresh_target_network_freq":refresh_target_network_freq,
                        "buffer_len": buffer_len,
                        "tmax":tmax, 
                        "agent":str(agent.network) 
                        }
def save_hyperparameter(dict,directory):    
    with open(directory+"/hyperparameters.json", "w") as outfile:
        json.dump(hyperparameters_train, outfile)

In [21]:
env.renderize=False
if TRAIN:
    
    state = env.reset()
    tb.add_graph(agent.network, torch.tensor(state, device=device, dtype=torch.float32))
    save_hyperparameter(hyperparameters_train,RES_DIR)
    loss_min=np.inf
    
    for step in trange(total_steps + 1):
        
        
        # reduce exploration as we progress
        agent.epsilon = rbt.epsilon_schedule(start_epsilon, end_epsilon, step, eps_decay_final_step)

        # take timesteps_per_epoch and update experience replay buffer
        _, state = rbt.play_and_record(state, agent, env, exp_replay, timesteps_per_epoch)

        
        # train by sampling batch_size of data from experience replay
        states, actions, rewards, next_states, done_flags, weights, idxs = exp_replay.sample(batch_size)
        actions =[agent.get_action_index(i) for i in actions]
        

        # loss = <compute TD loss>

        loss = rbt.compute_td_loss_priority_replay(agent, target_network, exp_replay,
                           states, actions, rewards, next_states, done_flags, weights, idxs,              
                           gamma=0.99,
                           device=device)
        if loss<loss_min:
            torch.save(agent.state_dict(), RES_DIR+'/best-model.pt')
         
        tb.add_scalar("1/TD Loss", loss, step)
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
        opt.step()
        opt.zero_grad()
        
        tb.add_scalar("2/Epsilon", agent.epsilon,step)

        if step % refresh_target_network_freq == 0:
            # Load agent weights into target_network
            target_network.load_state_dict(agent.state_dict())

        if step % eval_freq == 0:
            # eval the agent
            m_reward=rbt.evaluate(env, agent, n_games=50, greedy=True, t_max=tmax)[0] 
            tb.add_scalar("1/Mean reward per episode", m_reward, step)
            clear_output(True)
            print("buffer size = %i, epsilon = %.5f" %
                (len(exp_replay), agent.epsilon))
            print(f"Frequency evaluation = {eval_freq}")

    torch.save(agent.state_dict(), RES_DIR+'/last-model.pt')        
    tb.close()

  0%|          | 0/40001 [00:00<?, ?it/s]

In [ ]:
env.renderize=True
final_score,m_steps,infos = rbt.evaluate(env,agent, n_games=1, greedy=True, t_max=300)
print(f'final score:{final_score} in {m_steps} steps')
print(infos)
print(f'Well done , Distance: {env.distance()}, Fitness: {env.fitness()}')
print(f"collision {env.detect_collision()[0]}")

final score:363.72938874160013 in 299.0 steps
[['', '']]
Well done , Distance: 0.611573877831515, Fitness: 0.014699999999999842
collision False


In [ ]:
SAVE_AGENT=False
SAVE_AGENT=False

FILE_PREFIX=['mean_rw_history','td_loss_history']
EXTENSION='.pickle'
NAME="-reduced_network"
FILENAME=[]
for i in FILE_PREFIX:
    FILENAME.append(i+NAME+EXTENSION)


if SAVE_AGENT:
    torch.save(agent.state_dict(), 'model_panda_trained.pth')
    for (i,j) in zip(FILENAME,FILE_PREFIX):
        with open(i, 'wb') as handle:
            pickle.dump(j, handle, protocol=pickle.HIGHEST_PROTOCOL)
   

In [ ]:
state = env.reset()
qvalues = agent.get_qvalues([state])
action = agent.actions_space[qvalues.argmax(axis=-1)[0]]
state, r, done, info = env.step(action)
print(state, r, done, info)

[ 0.15 -0.55  3.2 ] -20.706927372722966 False ['', '']


In [ ]:
qvalues = agent.get_qvalues([state])
action = agent.actions_space[qvalues.argmax(axis=-1)[0]]
state, r, done, _ = env.step(action)

In [ ]:
state = env.reset()
reward = 0
while True:
    qvalues = agent.get_qvalues([state])
    action = agent.actions_space[qvalues.argmax(axis=-1)[0]]
    state, r, done, info = env.step(action)
    reward += r
    #print(reward)
    if done or reward < -60:
        print('Got reward: {}'.format(reward))
        break
print(f'Done , Distance: {env.distance()}')
print(f"collision {env.detect_collision()[0]}")
print(info)

Got reward: -62.69144567753095
Done , Distance: 0.6163060183056325
collision False
['', '']


In [ ]:
state=env.get_q()
qvalues = agent.get_qvalues([state])
qvalues

array([[339.97763, 352.16226, 341.89145, 349.60864, 346.29236, 349.448  ,
        352.5781 , 350.36682, 354.35278, 351.50418, 355.7459 , 350.91345,
        358.81012, 357.651  , 356.27682, 351.90414, 357.14294, 312.89474,
        347.17572, 345.7454 , 350.4716 , 351.26053, 357.87143, 354.52103,
        357.70673, 347.29922]], dtype=float32)

**Let us record a video of trained agent**

In [ ]:
# # Animate learned policy
# save_dir='./videos/'
# #env = make_env(env_name)
# generate_animation(env, agent, save_dir=save_dir)
# [filepath] = glob.glob(os.path.join(save_dir, '*.mp4'))

# display_animation(filepath)

In [ ]:
#TODO

# Run this on another environment in OpenAI Gym
# Create a robotic environment with more actions
#